In [ ]:
import numpy as np

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
def open_csv(path):

  d = spark.read.format("csv")\
    .option("header","false")\
    .option("inferSchema","true")\
    .load(path)

  return d

In [ ]:
train_csv = open_csv("dbfs:/FileStore/tables/emnist_balanced_train-3144f.csv")
test_csv = open_csv("dbfs:/FileStore/tables/emnist_balanced_test-dc5d0.csv")
train_csv = train_csv.withColumnRenamed('_c0', 'label')
test_csv = test_csv.withColumnRenamed('_c0', 'label')

In [ ]:
assembler = VectorAssembler(inputCols=train_csv.columns[1:],outputCol="features")
train = assembler.transform(train_csv)
test = assembler.transform(test_csv)

In [ ]:
layers = [784, 200, 100, 47]
trainer = MultilayerPerceptronClassifier(maxIter=120, layers=layers, blockSize=128, seed=1234)
model = trainer.fit(train)

In [ ]:
model.save("/MPmodel")

In [ ]:
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("accuracy:" + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
display(predictionAndLabels)